In [1]:
from read_in_data_functions import read_etf_returns
from stock_data_functions import TickerComparison, TickerData, calc_log_rets
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt


In [4]:
from matplotlib.pyplot import plot
import factor_analysis_functions
import stock_data_functions
from stock_data_functions import TickerData, TickerComparison
import importlib

import stock_data_functions
importlib.reload(factor_analysis_functions)
importlib.reload(stock_data_functions)

from factor_analysis_functions import combine_regressor_stock, get_session_and_trade_date, align_beta_stock, rolling_r2_from_intraday, combine_minute_returns, mask_initial_window_with_zeros, ewma_beta, run_full_pipeline, plot_interactive_r2_dashboard_5m

#### Manual Rolling R^2 Value

Read in

In [9]:
from stock_data_functions import get_top_N_US_stocks, TickerComparison

In [ ]:
warmup = 30 # Number of Days
regressor = ['SPY', 'QQQ']
regressor_ticker = 'SPY'
stock = ['META', 'MU', 'AMD', 'NVDA', 'AAPL', 'MSFT', 'INTC', 'ALAB', 'CRDO', 'TER', 'AMKR']
lam = 0.94
lookback_days = 20 # For rolling r2 

stock_object_minute = TickerComparison(stock, filing_date_gte='2023-11-01', period = 'minute', date_updated = True, fetch_in_chunks=True, waiting_time = 60, chunksize = 200)
regressor_object_minute = TickerComparison(regressor, filing_date_gte='2023-11-01', period = 'minute', date_updated = False)

stock_object_daily = TickerComparison(stock, filing_date_gte='2023-11-01', period = 'day', date_updated = True)
regressor_object_daily = TickerComparison(regressor, filing_date_gte='2023-11-01', period = 'day', date_updated = True)



Get log returns of your stock

In [227]:
stock_returns       = stock_object_daily.tickers_stocks_prices.copy().loc[:,(slice(None), 'close')].apply(calc_log_rets)
stock_returns       = stock_returns.droplevel(1, axis=1)

regressor_returns   = regressor_object_daily.tickers_stocks_prices.copy().loc[:,(slice(None), 'close')].apply(calc_log_rets)
regressor_returns   = regressor_returns.droplevel(1, axis=1)

In [228]:
stock_returns_min       = stock_object_minute.tickers_stocks_prices.copy().loc[:,(slice(None), 'close')].apply(calc_log_rets)
stock_returns_min       = stock_returns_min.droplevel(1, axis=1)
regressor_returns_min   = regressor_object_minute.tickers_stocks_prices.copy().loc[:,(slice(None), 'close')].apply(calc_log_rets)
regressor_returns_min   = regressor_returns_min.droplevel(1, axis=1)
regressor_rets_1min     = regressor_returns_min[regressor_ticker]

Get the returns corresponding to each trading session (close to close). This uses 5 min VWAP 

In [229]:
combined_returns_min = combine_minute_returns(
        stock_returns_min,
        regressor_rets_1min
    )

combined_returns_min.head()

,MSFT,MU,SPY
2023-11-01 04:30:00-04:00,0.000059,NaN,0.000767
2023-11-01 06:00:00-04:00,-0.000564,NaN,-0.000168
2023-11-01 06:05:00-04:00,0.000654,NaN,-0.000192
2023-11-01 06:10:00-04:00,-0.000594,NaN,-0.000288
2023-11-01 06:20:00-04:00,0.000594,NaN,-0.000240


Get EWMA beta on daily returns - must first align the daily series together then pass to EWMA function

In [268]:
betas_df = stock_returns.apply(
    ewma_beta,factor_ret=regressor_returns[regressor_ticker], lam=0.94)



Align beta dataframe with returns of regressor and stock - drop any where it doesn't exist (doesn't combine)

In [269]:
labelled_combined_returns_min = get_session_and_trade_date(combined_returns_min)

stock_returns_min_aligned     = combined_returns_min.drop(labels=[regressor_ticker], axis=1)
labelled_combined_returns_min = align_beta_stock(labelled_combined_returns_min, betas_df)

/Users/phillip/Desktop/Moon2/factor_analysis_functions.py:267: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  good_dates = stock_days[stock_days.isin(beta_days)]


In [270]:
r2_df = rolling_r2_from_intraday(labelled_combined_returns_min, betas_df, lookback_days=20, factor_col = regressor_ticker)

stock_cols :  Index(['MSFT', 'MU'], dtype='object')


There might be leading 0s and whatever when you combine everything - burn in period for beta of warm up

#### Plotly

In [27]:
import factor_analysis_functions
from factor_analysis_functions import plot_interactive_r2_dashboard_5m
import importlib
importlib.reload(factor_analysis_functions)
importlib.reload(stock_data_functions)

<module 'stock_data_functions' from '/Users/phillip/Desktop/Moon2/stock_data_functions.py'>

In [ ]:
warmup               = 30 # Number of Days
regressor            = ['SPY', 'I:NDX']
regressor_ticker     = 'SPY'
stock                = ['META', 'MU', 'AMD', 'NVDA','ALAB', 'CRDO', 'TER', 'AMKR']
lam                  = 0.94
short_lookback       = 5
long_lookback        = 20

fig = plot_interactive_r2_dashboard_5m(
                                    filing_date_gte='2023-09-01',
                                    stock=stock,
                                    regressor=regressor,
                                    init_stock=stock[0],
                                    init_regressor='I:NDX',
                                    short_lookback=short_lookback,
                                    long_lookback=long_lookback,
                                    lam=0.96,
                                    minute_fetch_in_chunks=True
                                    )



In [49]:
import plotly.io as pio
pio.renderers.default = "browser"
fig.show()

# Orthogonal vs Ridge EWMA Beta Model

#### Introduction to Beta Tuning Functions

Testing ridge vs orthogonal regression methods
- See which method yields the best results 
- A good beta model : beta doesn't fluctuate much, weights are interpretable, use MSEs 

In [734]:
import importlib
import factor_analysis_functions
from factor_analysis_functions import ewma_beta_diagnostic, ewma_beta
from stock_data_functions import TickerComparison, calc_log_rets
import datetime as dt
from pandas.tseries.offsets import BDay
from typing import List, Optional, Any
importlib.reload(factor_analysis_functions)

<module 'factor_analysis_functions' from '/Users/phillip/Desktop/Moon2/factor_analysis_functions.py'>

In [927]:
import ewma_beta_tuning
importlib.reload(ewma_beta_tuning)
from ewma_beta_tuning import _extract_daily_close_rets, prepare_ewma_inputs_from_tickers, compute_ewma_betas_from_inputs,ewma_betas_from_tickers


In [ ]:
regressors                  = ['SPY', 'SMH', 'I:NDX']
tickers                     = ['AMD', 'NVDA','INTC', 'ALAB', 'CRDO', 'TER']
daily_start_date            = (dt.datetime.today() - pd.Timedelta(weeks=104)).strftime('%Y-%m-%d')
lam                         = 0.94
date_updated_regressor      = False
date_updated                = False

In [948]:
# 1) Build daily inputs exactly like run_full_pipeline, but with your new helper
prep = prepare_ewma_inputs_from_tickers(
    stock=tickers,                   # your stocks
    regressors=regressors,             # any factor set you choose
    filing_date_gte="2023-09-01",
    date_updated=date_updated,
    date_updated_regressor=date_updated_regressor,
    daily_start_date=daily_start_date,
)

# 2) For each stock, run the diagnostic (ridge vs orthogonalize)
summaries = {}
betas = {}
for s in prep["stock_returns_daily"].columns:
    diag = ewma_beta_diagnostic(
        asset_ret=prep["stock_returns_daily"][s],
        factor_ret=prep["factor_returns_daily"],
        lam=lam,
        ridge=1e-6,                        # tiny stabilizer; set 0.0 for pure EWMA-OLS
        orth_order=regressors,         # order for orthogonalize; optional
    )
    summaries[s] = diag["summary"]         # table comparing both methods
    betas[s] = diag["betas"]               # {'ridge': DataFrame, 'orthogonalize': DataFrame}


[1/6] AMD: start
[1/6] AMD: done
[2/6] NVDA: start
[2/6] NVDA: done
[3/6] INTC: start
[3/6] INTC: done
[4/6] ALAB: start
[4/6] ALAB: done
[5/6] CRDO: start
[5/6] CRDO: done
[6/6] TER: start
[6/6] TER: done
[1/3] SPY: start
[1/3] SPY: done
[2/3] SMH: start
[2/3] SMH: done
[3/3] I:NDX: start
[3/3] I:NDX: done


In [206]:
print(f'Exp. Weighting {lam}')
print(f'Order : {regressors}\n')
for i in summaries.keys():
    print('------------------------------------------------------------')
    print(i,':',summaries[i])
    print('------------------------------------------------------------')
    print('StD of Ridge Betas: {}'.format(betas[i]['ridge'][30:].std().values.round(2)))
    print('StD of Ortho Betas: {}'.format(betas[i]['orthogonalize'][30:].std().values.round(2)))
    print('------------------------------------------------------------')


Exp. Weighting 0.94
Order : ['SPY', 'SMH', 'I:NDX']

------------------------------------------------------------
AMD :                pseudo_OOS_MSE  beta_roughness  nan_frac  factor_cond_num
ridge                0.000547        0.339555  0.038685       166.148986
orthogonalize        0.001963        0.521986  0.038685       166.148986
------------------------------------------------------------
StD of Ridge Betas: [1.26 0.57 1.1 ]
StD of Ortho Betas: [0.51 0.55 1.49]
------------------------------------------------------------
------------------------------------------------------------
NVDA :                pseudo_OOS_MSE  beta_roughness  nan_frac  factor_cond_num
ridge                0.000294        0.211663  0.038685       166.148986
orthogonalize        0.001847        1.050394  0.038685       166.148986
------------------------------------------------------------
StD of Ridge Betas: [0.9  0.45 0.74]
StD of Ortho Betas: [0.44 0.4  0.93]
-------------------------------------------

##### Picking the best parameters 
- Choose factor sets
- Choose ridge parameter
- Choose lam factor
- Choose method (ortho vs ridge)

Use the tune_betas_for_tickers to get the
- Grid for each combination

Call best_beta_method_per_stock
- Finds the best set of parameters for each ticker
- returns a dict[stock] = parameters

In [966]:
import ewma_beta_tuning
from ewma_beta_tuning import tune_betas_for_tickers, best_beta_method_per_stock, get_optimal_betas
importlib.reload(ewma_beta_tuning)


factor_sets = [["SPY"],
               ["SPY","SMH"],
               ["I:NDX","SMH"],
               ["SPY","SMH","I:NDX"],
               ['SMH'],
               ['I:NDX']]

ridge_grid  = [1e-8, 1e-6, 1e-4, 1e-3]
lambdas     = [0.90, 0.94, 0.96, 0.98, 0.99]
thresh      = 0.05 # Threshold for parameter tuning where e.g., 10% difference in pseudo OOS MSE is treated as the same

In [967]:

best, grid = tune_betas_for_tickers(
    asset_ret_by_ticker=prep['stock_returns_daily'],
    factor_ret_all=prep['factor_returns_daily'],
    factor_sets=factor_sets,
    lambdas=lambdas,
    ridge_grid=ridge_grid,
    methods=("ridge","orthogonalize"),
    min_obs=60,
    verbose=True,
)

best_params = best_beta_method_per_stock(grid, tickers, thresh)

[tune] AMD ...
[tune] NVDA ...
[tune] INTC ...
[tune] ALAB ...
[tune] CRDO ...
[tune] TER ...
ridge
0.05 0.2741159637019377 thresh, rel
ridge
0.05 0.8173686736465593 thresh, rel
ridge
0.05 0.0 thresh, rel
0.96
0.05 0.026848904375113485 thresh, rel
0.99
0.05 0.9366910557982118 thresh, rel
                          pseudo_oos_mse  beta_roughness  factor_cond_num
lam  factors      ridge                                                  
0.99 (I:NDX, SMH) 0.0001        0.000552        0.000389         26.68041
ridge
0.05 0.4883647701089544 thresh, rel
ridge
0.05 0.8955226134409576 thresh, rel
ridge
0.05 0.0 thresh, rel
0.96
0.05 0.029785888716064263 thresh, rel
0.99
0.05 0.8918255908134235 thresh, rel
                            pseudo_oos_mse  beta_roughness  factor_cond_num
lam  factors      ridge                                                    
0.99 (I:NDX, SMH) 0.000001        0.000278        0.006486         26.68041
ridge
0.05 0.13003178098758034 thresh, rel
ridge
0.05 0.8210944984

Get the optimal betas therefore

In [922]:
optimal_betas = {}
for i in best_params.keys():
    beta_df = ewma_betas_from_tickers(
        stock = [i],
        regressors = best_params[i]['best_factor'],
        filing_date_gte=daily_start_date,
        date_updated=date_updated,
        date_updated_regressor=date_updated_regressor,
        daily_start_date=daily_start_date,
        lam=best_params[i]['best_lam'],
        method=best_params[i]['best_method'],
        ridge=best_params[i]['best_ridge'],
        orth_order=best_params[i]['best_factor']
    )

    optimal_betas[i] = beta_df['betas_by_stock'][i]

[1/1] AMD: start
[1/1] AMD: done
[1/1] I:NDX: start
[1/1] I:NDX: done
[1/1] NVDA: start
[1/1] NVDA: done
[1/1] I:NDX: start
[1/1] I:NDX: done
[1/1] INTC: start
[1/1] INTC: done
[1/1] I:NDX: start
[1/1] I:NDX: done
[1/1] ALAB: start
[1/1] ALAB: done
[1/1] I:NDX: start
[1/1] I:NDX: done
[1/1] CRDO: start
[1/1] CRDO: done
[1/1] I:NDX: start
[1/1] I:NDX: done
[1/1] TER: start
[1/1] TER: done
[1/1] I:NDX: start
[1/1] I:NDX: done


#### Pipeline for optimal betas

In [ ]:
regressors                  = ['SPY', 'SMH', 'I:NDX']
tickers                     = ['AMD', 'NVDA','INTC', 'ALAB', 'CRDO', 'TER']
daily_start_date            = (dt.datetime.today() - pd.Timedelta(weeks=104)).strftime('%Y-%m-%d')
lam                         = 0.94
date_updated_regressor      = False
date_updated                = False

factor_sets = [["SPY"],
               ["SPY","SMH"],
               ["I:NDX","SMH"],
               ["SPY","SMH","I:NDX"],
               ['SMH'],
               ['I:NDX']]

ridge_grid  = [1e-8, 1e-6, 1e-4, 1e-3]
lambdas     = [0.90, 0.94, 0.96, 0.98, 0.99]
thresh      = 0.1                       # Threshold for parameter tuning where e.g., 10% difference in pseudo OOS MSE is treated as the same



In [975]:
import ewma_beta_tuning
from ewma_beta_tuning import get_optimal_betas
importlib.reload(ewma_beta_tuning)

optimal_betas, best_params = get_optimal_betas(tickers, regressors, factor_sets,
                                  ridge_grid, lambdas, thresh,
                                  date_updated,date_updated_regressor,
                                  daily_start_date)

[1/6] AMD: start
[1/6] AMD: done
[2/6] NVDA: start
[2/6] NVDA: done
[3/6] INTC: start
[3/6] INTC: done
[4/6] ALAB: start
[4/6] ALAB: done
[5/6] CRDO: start
[5/6] CRDO: done
[6/6] TER: start
[6/6] TER: done
[1/3] SPY: start
[1/3] SPY: done
[2/3] SMH: start
[2/3] SMH: done
[3/3] I:NDX: start
[3/3] I:NDX: done
[tune] AMD ...
[tune] NVDA ...
[tune] INTC ...
[tune] ALAB ...
[tune] CRDO ...
[tune] TER ...
ridge
0.05 0.2741159637019377 thresh, rel
ridge
0.05 0.8173686736465593 thresh, rel
ridge
0.05 0.0 thresh, rel
0.96
0.05 0.026848904375113485 thresh, rel
0.99
0.05 0.9366910557982118 thresh, rel
ridge
0.05 0.4883647701089544 thresh, rel
ridge
0.05 0.8955226134409576 thresh, rel
ridge
0.05 0.0 thresh, rel
0.96
0.05 0.029785888716064263 thresh, rel
0.99
0.05 0.8918255908134235 thresh, rel
ridge
0.05 0.13003178098758034 thresh, rel
ridge
0.05 0.8210944984960646 thresh, rel
ridge
0.05 0.0 thresh, rel
0.99
0.05 0.04146471577759416 thresh, rel
0.99
0.05 0.8822310577357143 thresh, rel
ridge
0.05 0

#### intraday stuff

In [1330]:
nvda = TickerData('NVDA', filing_date_gte='2023-09-09')
nvda.get_historical_prices(date_updated=False)
nvda.get_ratios()
nvda.compute_rolling_return_zscores()
summary_data = nvda.summary_returns_data.copy()

2025-10-28
Call NVDA.calc_zscore_and_rets() to get returns data
found


In [1331]:
from stock_data_functions import TickerData
import stock_data_functions
importlib.reload(stock_data_functions)

from factor_analysis_functions import find_max_min_trade_date
importlib.reload(factor_analysis_functions)

<module 'factor_analysis_functions' from '/Users/phillip/Desktop/Moon2/factor_analysis_functions.py'>

In [1397]:
nvda_5m_prices                  = tickers.tickers_stocks_prices['NVDA'].copy()
nvda_5m_prices_w_               = get_session_and_trade_date(nvda_5m_prices)
nvda_5m_prices_w_['trade_date'] = pd.to_datetime(nvda_5m_prices_w_['trade_date'])
df = nvda_5m_prices_w_.groupby('trade_date').apply(find_max_min_trade_date).dropna(subset=['high','low'], how='any')

df['intraday_max_rets'] = np.where(df['increase'] == True,
                                   np.log((df['high']/df['low'])),
                                   np.log(df['low']/df['high']))

df = df.droplevel(1, axis=0)
df.index = pd.to_datetime(df.index)
merged = pd.merge(df, summary_data.dropna(subset=['zscore_low']), left_index=True, right_index=True, how='inner')


/var/folders/2_/q3fhp8js4zgd2w2m2cb4y8rh0000gn/T/ipykernel_85457/2187987264.py:4: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



#### Big Swing in Post Market

In [1327]:
from stock_data_functions import TickerData
import stock_data_functions
importlib.reload(stock_data_functions)

from factor_analysis_functions import find_max_min_trade_date
importlib.reload(factor_analysis_functions)

<module 'factor_analysis_functions' from '/Users/phillip/Desktop/Moon2/factor_analysis_functions.py'>

In [1452]:
dataset = ['NVDA', 'AMD', 'MU', 'INTC', 'AVGO']

In [ ]:
tickers = TickerComparison(dataset, filing_date_gte='2023-11-01', waiting_time=30, date_updated=True, period='minute', fetch_in_chunks=False)

In [1333]:
ht = merged.index.astype(str) + ' ' + merged['high_time'].astype(str)
ht = pd.to_datetime(ht)

lt = merged.index.astype(str) + ' ' + merged['low_time'].astype(str)
lt = pd.to_datetime(lt)

In [ ]:
def separate_post_pre_market_high_lows(merged):
    """ 
    Separate when the high and lows each day are in the post and pre market
    """
    merged['keep_ht'] = np.select(
        [
            (ht.dt.time >= dt.time(4, 0))  & (ht.dt.time < dt.time(9, 30)),   # Pre-Market
            (ht.dt.time >= dt.time(9, 30)) & (ht.dt.time < dt.time(16, 0)),   # Regular
            (ht.dt.time >= dt.time(16, 0)) & (ht.dt.time < dt.time(21, 0)),   # Post-Market
        ],
            ['Pre-Market', 'Regular', 'Post-Market'],
        default='closed'

    )
    merged['keep_lt'] = np.select(
        [
            (lt.dt.time >= dt.time(4, 0))  & (lt.dt.time < dt.time(9, 30)),   # Pre-Market
            (lt.dt.time >= dt.time(9, 30)) & (lt.dt.time < dt.time(16, 0)),   # Regular
            (lt.dt.time >= dt.time(16, 0)) & (lt.dt.time < dt.time(21, 0)),   # Post-Market
        ],
            ['Pre-Market', 'Regular', 'Post-Market'],
        default='closed'

    )
    post_market_pre_market = merged[(merged['keep_ht'] != 'Regular') | (merged['keep_lt'] != 'Regular')].copy()

    return post_market_pre_market

In [ ]:
keep = ['increase', 'high', 'low', 'high_time', 'low_time', 'high_volume', 'low_volume', 'intraday_max_rets','ret_vwap', 'retstd_vwap', 'zbin_vwap', 'zscore_high', 'zscore_low', 'zbin_high', 'zbin_low', 'zbin_high']


df = separate_post_pre_market_high_lows(merged)[keep]

#### See if it is idio

In [1354]:
results = plots['results']
r2_short_long = pd.DataFrame(results['NVDA']['metrics'].values(), results['NVDA']['metrics'].keys()).T
r2_short_long.drop(labels=['shapes'], inplace=True, axis=1)
r2_short_long.set_index(['x'], inplace=True)

nvda_results2 = plots['results']['NVDA']['df']

nvda_results2['beta'] = nvda_results2['trade_date'].map(plots['beta_df']['NVDA'])
nvda_results2['beta_rets'] = nvda_results2['NVDA'] * nvda_results2['beta']
nvda_results2['idio_rets'] = nvda_results2['NVDA'] - nvda_results2['beta_rets']

/Users/phillip/.pyenv/versions/3.10.13/lib/python3.10/site-packages/pandas/core/indexes/base.py:7588: FutureWarning:

Dtype inference on a pandas object (Series, Index, ExtensionArray) is deprecated. The Index constructor will keep the original dtype in the future. Call `infer_objects` on the result to get the old behavior.



In [1390]:
r2_short_long

,r2_short,r2_long,idio_flag,macro_flag
x,,,,
2023-11-09,-0.947849,-0.947849,False,False
2023-11-10,-0.258792,-0.258792,False,False
2023-11-13,-0.236003,-0.236003,False,False
2023-11-14,-0.028299,-0.028299,False,False
2023-11-15,-0.042099,-0.042099,False,False
...,...,...,...,...
2025-10-22,0.483126,0.327291,False,False
2025-10-23,0.414405,0.325432,False,False
2025-10-24,0.155404,0.329475,False,False


In [1352]:
nvda_results2

,regressor_rets,NVDA,trade_date,beta,beta_rets,idio_rets
2023-11-08 16:00:00+00:00,0.000840,0.002482,2023-11-09,1.175258,0.002917,-0.000435
2023-11-08 16:05:00+00:00,-0.000924,-0.001165,2023-11-09,1.175258,-0.001369,0.000204
2023-11-08 16:10:00+00:00,0.000525,0.003641,2023-11-09,1.175258,0.004279,-0.000638
2023-11-08 16:15:00+00:00,0.001322,-0.000204,2023-11-09,1.175258,-0.000240,0.000036
2023-11-08 16:20:00+00:00,-0.000861,-0.001278,2023-11-09,1.175258,-0.001503,0.000224
...,...,...,...,...,...,...
2025-10-27 23:35:00+00:00,0.000000,0.000209,2025-10-27,0.652481,0.000136,0.000073
2025-10-27 23:40:00+00:00,-0.000295,-0.000130,2025-10-27,0.652481,-0.000085,-0.000045
2025-10-27 23:45:00+00:00,0.000328,-0.000600,2025-10-27,0.652481,-0.000392,-0.000209
2025-10-27 23:50:00+00:00,0.000262,0.000313,2025-10-27,0.652481,0.000204,0.000109
